In [39]:
import pandas as pd
import json
import re
#from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [40]:
run_name = "eval_make_model_v2"

# load JSONL
records = []
with open("../aircraft_er_predictions/" + run_name + "_predictions_all.tsv") as f:
    for line in f:
        records.append(json.loads(line))



In [41]:
df = pd.DataFrame(records)
print(len(df))
df.head()

303065


,left,right,match,match_confidence
0,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Beech Bonanza 35A/C/D/E/G/...,0,0.999983
1,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Gipps Aero Ga8 Airvan\n,1,0.999639
2,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Cessna 180\n,1,0.987671
3,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Cessna 180A/B\n,1,0.997780
4,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Cessna 185A/B/C Skywagon\n,0,0.966812


In [42]:
df_raw = pd.read_csv("../data/aircraft_er/" + run_name + "/all_pairs_with_id.txt", sep="," ) #, names=["left_id", "right_id", "gold"])
print(len(df_raw))
df_raw.head()

303065


,left_id,right_id,left,right
0,AERMACCHI AL60,10,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Beech Bonanza 35A/C/D/E/G/...
1,AERMACCHI AL60,26,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Gipps Aero Ga8 Airvan
2,AERMACCHI AL60,30,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Cessna 180
3,AERMACCHI AL60,31,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Cessna 180A/B
4,AERMACCHI AL60,33,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Cessna 185A/B/C Skywagon


In [43]:
df["left_id"] = df_raw["left_id"] 
df["right_id"] = df_raw["right_id"] 


In [44]:
df.head()

,left,right,match,match_confidence,left_id,right_id
0,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Beech Bonanza 35A/C/D/E/G/...,0,0.999983,AERMACCHI AL60,10
1,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Gipps Aero Ga8 Airvan\n,1,0.999639,AERMACCHI AL60,26
2,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Cessna 180\n,1,0.987671,AERMACCHI AL60,30
3,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Cessna 180A/B\n,1,0.997780,AERMACCHI AL60,31
4,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Cessna 185A/B/C Skywagon\n,0,0.966812,AERMACCHI AL60,33


In [45]:
df["description"] = df["right"].str.extract(
    r"COL\s+description\s+VAL\s*(.*?)(?=\n\s*COL|$)",
    expand=False
)

In [46]:
print('all evaluated potential paris: ', len(df))
df_matches = df[df["match"]==1]
print('matching paris: ', len(df_matches))

all evaluated potential paris:  303065
matching paris:  201290


In [47]:
df_matches.head()

,left,right,match,match_confidence,left_id,right_id,description
1,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Gipps Aero Ga8 Airvan\n,1,0.999639,AERMACCHI AL60,26,Gipps Aero Ga8 Airvan
2,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Cessna 180\n,1,0.987671,AERMACCHI AL60,30,Cessna 180
3,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Cessna 180A/B\n,1,0.997780,AERMACCHI AL60,31,Cessna 180A/B
5,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Helio H-250/295/395\n,1,0.999956,AERMACCHI AL60,34,Helio H-250/295/395
7,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Cessna 172 Skyhawk\n,1,0.998517,AERMACCHI AL60,36,Cessna 172 Skyhawk


In [48]:
df_max = (
    df_matches
      .groupby(['right_id', 'description'], as_index=False)
      .agg({
          'match_confidence':      "max",
          'left_id':              lambda s: pd.unique(s).tolist(),
      })
      .rename(columns={'match_confidence': 'match_confidence_max', 'left_id': 'left_ids'})
      .assign(left_ids_count=lambda d: d['left_ids'].str.len())
)



In [49]:
df_max

,right_id,description,match_confidence_max,left_ids,left_ids_count
0,10,Beech Bonanza 35A/C/D/E/G/H/J/K/S/V/ 36A,0.999720,"[BEECH 35, BEECH 36, HUGHES 369, LEARJET 36, L...",5
1,26,Gipps Aero Ga8 Airvan,0.999990,"[AERMACCHI AL60, AERO COMMANDER 100, AERO COMM...",849
2,30,Cessna 180,0.999994,"[AERMACCHI AL60, AERO COMMANDER 100, AERO COMM...",675
3,31,Cessna 180A/B,0.999988,"[AERMACCHI AL60, AERO COMMANDER 100, AERO COMM...",637
4,33,Cessna 185A/B/C Skywagon,0.999988,"[AERO COMMANDER 200, AERO COMMANDER 520, AERO ...",171
...,...,...,...,...,...
240,882,Airbus Industrie A380-800,0.999923,"[AERO COMMANDER 520, AERO COMMANDER 681, AERO ...",199
241,887,B787-800 Dreamliner,0.999993,"[AERMACCHI AL60, AERMACCHI F260, AERO COMMANDE...",1041
242,888,Boeing 737-900ER,0.999994,"[AERMACCHI AL60, AERO COMMANDER 200, AERO COMM...",454
243,889,B787-900 Dreamliner,0.999992,"[AERMACCHI AL60, AERMACCHI F260, AERO COMMANDE...",1036


In [50]:
df_join = df_matches.merge(
    df_max.drop(columns=['description']),
    left_on="right_id",
    right_on="right_id",
    how='left'
)
df_join["is_max"] = df_join["match_confidence"]==df_join["match_confidence_max"]

In [51]:
df_join

,left,right,match,match_confidence,left_id,right_id,description,match_confidence_max,left_ids,left_ids_count,is_max
0,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Gipps Aero Ga8 Airvan\n,1,0.999639,AERMACCHI AL60,26,Gipps Aero Ga8 Airvan,0.999990,"[AERMACCHI AL60, AERO COMMANDER 100, AERO COMM...",849,False
1,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Cessna 180\n,1,0.987671,AERMACCHI AL60,30,Cessna 180,0.999994,"[AERMACCHI AL60, AERO COMMANDER 100, AERO COMM...",675,False
2,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Cessna 180A/B\n,1,0.997780,AERMACCHI AL60,31,Cessna 180A/B,0.999988,"[AERMACCHI AL60, AERO COMMANDER 100, AERO COMM...",637,False
3,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Helio H-250/295/395\n,1,0.999956,AERMACCHI AL60,34,Helio H-250/295/395,0.999993,"[AERMACCHI AL60, AERMACCHI F260, AERO COMMANDE...",1046,False
4,COL make VAL AERMACCHI COL model VAL AL60,COL description VAL Cessna 172 Skyhawk\n,1,0.998517,AERMACCHI AL60,36,Cessna 172 Skyhawk,0.999995,"[AERMACCHI AL60, AERO COMMANDER 100, AERO COMM...",720,False
...,...,...,...,...,...,...,...,...,...,...,...
201285,COL make VAL ZLIN COL model VAL Z526,COL description VAL Ilyushiin Il-96-400t\n,1,0.999976,ZLIN Z526,881,Ilyushiin Il-96-400t,0.999994,"[AERMACCHI AL60, AERMACCHI F260, AERO COMMANDE...",1056,False
201286,COL make VAL ZLIN COL model VAL Z526,COL description VAL B787-800 Dreamliner\n,1,0.999956,ZLIN Z526,887,B787-800 Dreamliner,0.999993,"[AERMACCHI AL60, AERMACCHI F260, AERO COMMANDE...",1041,False
201287,COL make VAL ZLIN COL model VAL Z526,COL description VAL Boeing 737-900ER\n,1,0.996571,ZLIN Z526,888,Boeing 737-900ER,0.999994,"[AERMACCHI AL60, AERO COMMANDER 200, AERO COMM...",454,False
201288,COL make VAL ZLIN COL model VAL Z526,COL description VAL B787-900 Dreamliner\n,1,0.999960,ZLIN Z526,889,B787-900 Dreamliner,0.999992,"[AERMACCHI AL60, AERMACCHI F260, AERO COMMANDE...",1036,False


In [52]:
df_join.to_csv("../aircraft_er_predictions/" + run_name + "_all_matches.csv", index=False)

In [53]:
df_join_max = df_join[df_join["is_max"]==True]

df_join_max

,left,right,match,match_confidence,left_id,right_id,description,match_confidence_max,left_ids,left_ids_count,is_max
9626,COL make VAL AEROSPATIALE COL model VAL AS350 ...,COL description VAL Eurocompter AS350 B2 AStar\n,1,0.999995,AEROSPATIALE AS350,340,Eurocompter AS350 B2 AStar,0.999995,"[AERMACCHI AL60, AERMACCHI F260, AERO COMMANDE...",859,True
9856,COL make VAL AEROSPATIALE COL model VAL AS350 ...,COL description VAL Eurocompter AS350 B2 AStar\n,1,0.999995,AEROSPATIALE AS350,340,Eurocompter AS350 B2 AStar,0.999995,"[AERMACCHI AL60, AERMACCHI F260, AERO COMMANDE...",859,True
12669,COL make VAL AGUSTA COL model VAL AW139,COL description VAL Agustawestland AW139\n,1,0.999996,AGUSTA AW139,393,Agustawestland AW139,0.999996,"[AERMACCHI AL60, AERMACCHI F260, AERO COMMANDE...",1014,True
15079,COL make VAL AIRBUS COL model VAL A300 COL nam...,COL description VAL Airbus Industrie A300B/C/F...,1,0.999992,AIRBUS A300,690,Airbus Industrie A300B/C/F-100/200,0.999992,"[AERO COMMANDER 100, AERO COMMANDER 200, AERON...",75,True
15080,COL make VAL AIRBUS COL model VAL A300 COL nam...,COL description VAL Airbus Industrie A300-600/...,1,0.999989,AIRBUS A300,691,Airbus Industrie A300-600/R/CF/RCF,0.999989,"[AERO COMMANDER 200, AERO COMMANDER 600S2D, AE...",87,True
...,...,...,...,...,...,...,...,...,...,...,...
167347,COL make VAL SIKORSKY COL model VAL SH3,COL description VAL McDonnell Douglas DC-8-72\n,1,0.999983,SIKORSKY SH3,862,McDonnell Douglas DC-8-72,0.999983,"[AERMACCHI AL60, AERMACCHI F260, AERO COMMANDE...",853,True
170508,COL make VAL STEARMAN COL model VAL 75,COL description VAL Bombardier Learjet 75\n,1,0.999961,STEARMAN 75,575,Bombardier Learjet 75,0.999961,"[AERMACCHI AL60, AERO COMMANDER 100, AERO COMM...",512,True
177250,COL make VAL TAYLORCRAFT COL model VAL BC,COL description VAL Cessna Citation II/ Bravo\n,1,0.999985,TAYLORCRAFT BC,636,Cessna Citation II/ Bravo,0.999985,"[AERMACCHI AL60, AERMACCHI F260, AERO COMMANDE...",798,True
177386,COL make VAL TAYLORCRAFT COL model VAL BC12,COL description VAL Cessna C-421\n,1,0.999972,TAYLORCRAFT BC12,121,Cessna C-421,0.999972,"[AERMACCHI AL60, AERO COMMANDER 200, AERO COMM...",439,True


In [54]:
df_join_max

,left,right,match,match_confidence,left_id,right_id,description,match_confidence_max,left_ids,left_ids_count,is_max
9626,COL make VAL AEROSPATIALE COL model VAL AS350 ...,COL description VAL Eurocompter AS350 B2 AStar\n,1,0.999995,AEROSPATIALE AS350,340,Eurocompter AS350 B2 AStar,0.999995,"[AERMACCHI AL60, AERMACCHI F260, AERO COMMANDE...",859,True
9856,COL make VAL AEROSPATIALE COL model VAL AS350 ...,COL description VAL Eurocompter AS350 B2 AStar\n,1,0.999995,AEROSPATIALE AS350,340,Eurocompter AS350 B2 AStar,0.999995,"[AERMACCHI AL60, AERMACCHI F260, AERO COMMANDE...",859,True
12669,COL make VAL AGUSTA COL model VAL AW139,COL description VAL Agustawestland AW139\n,1,0.999996,AGUSTA AW139,393,Agustawestland AW139,0.999996,"[AERMACCHI AL60, AERMACCHI F260, AERO COMMANDE...",1014,True
15079,COL make VAL AIRBUS COL model VAL A300 COL nam...,COL description VAL Airbus Industrie A300B/C/F...,1,0.999992,AIRBUS A300,690,Airbus Industrie A300B/C/F-100/200,0.999992,"[AERO COMMANDER 100, AERO COMMANDER 200, AERON...",75,True
15080,COL make VAL AIRBUS COL model VAL A300 COL nam...,COL description VAL Airbus Industrie A300-600/...,1,0.999989,AIRBUS A300,691,Airbus Industrie A300-600/R/CF/RCF,0.999989,"[AERO COMMANDER 200, AERO COMMANDER 600S2D, AE...",87,True
...,...,...,...,...,...,...,...,...,...,...,...
167347,COL make VAL SIKORSKY COL model VAL SH3,COL description VAL McDonnell Douglas DC-8-72\n,1,0.999983,SIKORSKY SH3,862,McDonnell Douglas DC-8-72,0.999983,"[AERMACCHI AL60, AERMACCHI F260, AERO COMMANDE...",853,True
170508,COL make VAL STEARMAN COL model VAL 75,COL description VAL Bombardier Learjet 75\n,1,0.999961,STEARMAN 75,575,Bombardier Learjet 75,0.999961,"[AERMACCHI AL60, AERO COMMANDER 100, AERO COMM...",512,True
177250,COL make VAL TAYLORCRAFT COL model VAL BC,COL description VAL Cessna Citation II/ Bravo\n,1,0.999985,TAYLORCRAFT BC,636,Cessna Citation II/ Bravo,0.999985,"[AERMACCHI AL60, AERMACCHI F260, AERO COMMANDE...",798,True
177386,COL make VAL TAYLORCRAFT COL model VAL BC12,COL description VAL Cessna C-421\n,1,0.999972,TAYLORCRAFT BC12,121,Cessna C-421,0.999972,"[AERMACCHI AL60, AERO COMMANDER 200, AERO COMM...",439,True


In [55]:
df_join_max.to_csv('temp.csv')

df_join_max.to_csv("../aircraft_er_predictions/" + run_name + "_max_matches.csv", index=False)

In [56]:
df_matches[df_matches["right"].isna()]


,left,right,match,match_confidence,left_id,right_id,description
